In [1]:
using Pkg
Pkg.add(["NCDatasets", "Statistics", "StatsBase", "HypothesisTests", "Dates", "ProgressMeter", "Plots", "DataFrames", "Distributions"])

   Resolving package versions...
  No Changes to `C:\Users\arikt\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\arikt\.julia\environments\v1.10\Manifest.toml`


In [ ]:
# Required packages
using NCDatasets
using Statistics
using Dates
using DataFrames
using StatsBase
using Plots
using ProgressMeter
using HypothesisTests
using Printf

"""
Process NetCDF models with chunked latitude processing
"""
function process_models_chunked_lat(directory_path::String, rcp_value::String, return_period_value::String, set_num_chunks::Integer)
    # List all files in the directory
    all_files = readdir(directory_path, join=true)
    println("Total files found in directory: ", length(all_files))
    
    # Filter for .nc4 files
    nc4_files = filter(f -> endswith(f, ".nc4"), all_files)
    println("Files with .nc4 extension: ", length(nc4_files))
    
    # Split each filename by underscore and check the specific positions
    matching_files = filter(nc4_files) do f
        parts = split(basename(f), "_")
        # Check if we have enough parts and the specific positions match
        length(parts) ≥ 9 && 
        parts[5] == rcp_value && 
        parts[9] == return_period_value
    end
    
    println("Files matching RCP '$(rcp_value)' and return period '$(return_period_value)': ", length(matching_files))
    if !isempty(matching_files)
        println("Example matching file: ", basename(matching_files[1]))
    end
    
    isempty(matching_files) && error("No files found matching criteria. Please check the directory path and file naming pattern.")
 
        
    # Get dimensions from first file
    ds = Dataset(matching_files[1])
    time_var = ds["time"][:]  # Changed from 'time' to 'time_var'
    lat = ds["lat"][:]
    lon = ds["lon"][:]
    close(ds)

    num_models = length(matching_files)
    num_time = length(time_var)  # Changed from 'time' to 'time_var'
    num_lat = length(lat)
    num_lon = length(lon)

    # Initialize result array
    result = Array{Float64}(undef, num_lat, num_lon, 5, num_models)

    # Set chunk size
    chunk_size_lat = ceil(Int, num_lat / set_num_chunks)

    # Process each model
    @showprogress "Processing models" for (model_idx, file) in enumerate(matching_files)
        println("Processing model $model_idx of $num_models: $(basename(file))")
        ds = Dataset(file)
        
        # Process in latitude chunks
        lat_indices = 1:chunk_size_lat:num_lat
        
        @showprogress "Processing latitude chunks" for (chunk_idx, lat_start) in enumerate(lat_indices)
            lat_end = min(lat_start + chunk_size_lat - 1, num_lat)
            lat_count = lat_end - lat_start + 1
            
            # Calculate total iterations for this chunk
            total_iterations = lat_count * num_lon
            iteration_counter = 0
            last_update_time = time_ns()  # Using time_ns() instead of time()
            update_interval = 300 * 1e9  # 300 seconds in nanoseconds
            
            # Read chunk data
            flddph_chunk = permutedims(
                ds["flddph"][:, lat_start:lat_end, :],
                [2, 1, 3]
            )
            
            
            # Process each grid point in chunk
            for i in 1:lat_count
                for j in 1:num_lon
                    iteration_counter += 1
                    
                    # Process data
                    data_ts = flddph_chunk[i, j, :]
                    
                    # Convert Missing to Float64 and handle NaN
                    data_ts_float = Float64[ismissing(x) ? NaN : Float64(x) for x in data_ts]
                    valid_idx = findall(!isnan, data_ts_float)
                    valid_data = data_ts_float[valid_idx]
                    
                    if length(valid_data) >= 2
                        # Calculate quantiles regardless of uniqueness
                        first_40 = valid_data[1:min(40, length(valid_data))]
                        result[lat_start+i-1, j, 1, model_idx] = quantile(first_40, 0.25)
                        result[lat_start+i-1, j, 2, model_idx] = quantile(first_40, 0.50)
                        result[lat_start+i-1, j, 3, model_idx] = quantile(first_40, 0.75)
                        
                        if length(unique(valid_data)) > 1
                            # Calculate slope and correlation only if there's variability
                            slopes = Float64[]
                            for i1 in 1:length(valid_idx)
                                for i2 in (i1+1):length(valid_idx)
                                    push!(slopes, (valid_data[i2] - valid_data[i1]) / (valid_idx[i2] - valid_idx[i1]))
                                end
                            end
                            result[lat_start+i-1, j, 4, model_idx] = median(slopes)
                            
                            # Correlation test
                            test = CorrelationTest(Float64.(valid_idx), valid_data)
                            result[lat_start+i-1, j, 5, model_idx] = pvalue(test)
                        else
                            # Set slope and p-value to NaN when no variability
                            result[lat_start+i-1, j, 4:5, model_idx] .= NaN
                        end
                        
                        # Print status update every 5 minutes
                        current_time = time_ns()
                        if current_time - last_update_time > update_interval
                            progress = iteration_counter / total_iterations * 100
                            println("\nStatus Update:")
                            println("Processing model $model_idx of $num_models: $(basename(file))")
                            @printf("Progress: %.2f%%\n", progress)
                            println("Current values at position ($i, $j):")
                            println("  25th percentile: $(result[lat_start+i-1, j, 1, model_idx])")
                            println("  Median: $(result[lat_start+i-1, j, 2, model_idx])")
                            println("  75th percentile: $(result[lat_start+i-1, j, 3, model_idx])")
                            println("  Theil-Sen slope: $(result[lat_start+i-1, j, 4, model_idx])")
                            println("  Correlation p-value: $(result[lat_start+i-1, j, 5, model_idx])\n")
                            last_update_time = current_time
                        end
                    else
                        # Handle cases with insufficient data
                        result[lat_start+i-1, j, :, model_idx] .= NaN
                    end
                end
            end            
            GC.gc() # Garbage collection
        end
        close(ds)
    end    
    return result, lat, lon
end

"""
Save results to NetCDF file
"""
function save_to_netcdf(result, lat, lon, output_path, rcp_value, return_period_value)
    ds = Dataset(output_path, "c")
    
    # Define dimensions
    defDim(ds, "lat", length(lat))
    defDim(ds, "lon", length(lon))
    defDim(ds, "value", 5)
    defDim(ds, "model", size(result, 4))
    
    # Define variables
    v = defVar(ds, "result", Float64, ("lat", "lon", "value", "model"))
    v[:,:,:,:] = result
    
    # Add attributes
    ds.attrib["title"] = "Results Array Containing Statistical Analysis"
    ds.attrib["creation_date"] = string(now())
    
    close(ds)
end

# Main execution
function main()
    nc_in_path = "C:\\ClimateData\\Flood\\FloodDepth\\Future\\rawNcs\\"
    nc_out_path = "C:\\ClimateData\\Flood\\FloodDepth\\Reanalysis\\"
    rcp_value = "rcp26"
    return_period_value = "none"
    set_num_chunks = 2
    
    start_time = now()
    
    # Process data
    result, lat, lon = process_models_chunked_lat(nc_in_path, rcp_value, return_period_value, set_num_chunks)
    
    # Save results
    output_file = joinpath(nc_out_path, "results_$(rcp_value)_$(return_period_value).nc4")
    save_to_netcdf(result, lat, lon, output_file, rcp_value, return_period_value)
    
    println("Time elapsed: ", now() - start_time)
end

# Run the program
main()

Total files found in directory: 216
Files with .nc4 extension: 216
Files matching RCP 'rcp26' and return period 'none': 24
Example matching file: cama-flood_clm45_gfdl-esm2m_ewembi_rcp26_2005soc_co2_flddph_none_150arcsec_global_annual_2006_2100.nc4
Processing model 1 of 24: cama-flood_clm45_gfdl-esm2m_ewembi_rcp26_2005soc_co2_flddph_none_150arcsec_global_annual_2006_2100.nc4

Status Update:
Progress: 27.16%
Current values at position (587, 5819):
  25th percentile: 1.2947204113006592
  Median: 1.5828269720077515
  75th percentile: 1.8563411235809326
  Theil-Sen slope: 0.0005536675453186035
  Correlation p-value: 0.7733253729513448


Status Update:
Progress: 37.65%
Current values at position (814, 1724):
  25th percentile: 1.5640988051891327
  Median: 2.0937626361846924
  75th percentile: 2.410723328590393
  Theil-Sen slope: -0.002126709620157878
  Correlation p-value: 0.16461797406667616


Status Update:
Progress: 51.20%
Current values at position (1106, 7488):
  25th percentile: 0.0
 

Processing models   8%|███                               |  ETA: 11:11:40


Processing model 3 of 24: cama-flood_clm45_ipsl-cm5a-lr_ewembi_rcp26_2005soc_co2_flddph_none_150arcsec_global_annual_2006_2100.nc4

Status Update:
Progress: 27.64%
Current values at position (598, 779):
  25th percentile: 1.2884356081485748
  Median: 1.8435452580451965
  75th percentile: 2.1848148107528687
  Theil-Sen slope: -0.004626210778951645
  Correlation p-value: 0.08316153550946599


Status Update:
Progress: 38.97%
Current values at position (842, 6997):
  25th percentile: 0.0
  Median: 0.0
  75th percentile: 0.0
  Theil-Sen slope: 0.0
  Correlation p-value: 0.13567530269897568


Status Update:
Progress: 54.11%
Current values at position (1169, 6074):
  25th percentile: 1.2917028367519379
  Median: 1.459502935409546
  75th percentile: 2.5487624406814575
  Theil-Sen slope: 0.0002843088963452507
  Correlation p-value: 0.45098935815403474


Status Update:
Progress: 85.12%
Current values at position (1839, 5096):
  25th percentile: 2.5378255248069763
  Median: 3.2033791542053223
  

Processing models  12%|█████                             |  ETA: 10:35:32


Processing model 4 of 24: cama-flood_clm45_miroc5_ewembi_rcp26_2005soc_co2_flddph_none_150arcsec_global_annual_2006_2100.nc4

Status Update:
Progress: 27.73%
Current values at position (599, 7607):
  25th percentile: 0.28754865378141403
  Median: 0.414845734834671
  75th percentile: 0.6994829475879669
  Theil-Sen slope: -0.000311345106265584
  Correlation p-value: 0.6233001115522607


Status Update:
Progress: 39.27%
Current values at position (849, 2577):
  25th percentile: 0.8904915601015091
  Median: 1.1444984674453735
  75th percentile: 1.230663388967514
  Theil-Sen slope: 0.0004987301451436589
  Correlation p-value: 0.8133515339485956


Status Update:
Progress: 54.87%
Current values at position (1186, 1651):
  25th percentile: 0.2563582956790924
  Median: 0.2709371894598007
  75th percentile: 0.4151475802063942
  Theil-Sen slope: -3.352951496205431e-5
  Correlation p-value: 0.4308574949617028


Status Update:
Progress: 86.73%
Current values at position (1874, 2589):
  25th percent

...

Processing models  71%|█████████████████████████         |  ETA: 3:32:14

Processing model 18 of 24: cama-flood_matsiro_hadgem2-es_ewembi_rcp26_2005soc_co2_flddph_none_150arcsec_global_annual_2006_2100.nc4

Status Update:
Progress: 27.50%
Current values at position (594, 8369):
  25th percentile: 1.9335188567638397
  Median: 2.602865695953369
  75th percentile: 2.925174593925476
  Theil-Sen slope: -0.000656174075218939
  Correlation p-value: 0.6953625136864814


Status Update:
Progress: 38.50%
Current values at position (832, 4614):
  25th percentile: 0.0
  Median: 0.031190693378448486
  75th percentile: 0.5597578883171082
  Theil-Sen slope: 0.0
  Correlation p-value: 0.13548100614843114


Status Update:
Progress: 54.85%
Current values at position (1185, 7281):
  25th percentile: 2.234959602355957
  Median: 3.084273934364319
  75th percentile: 3.540600061416626
  Theil-Sen slope: 0.0049920383887954905
  Correlation p-value: 0.4350208677285736


Status Update:
Progress: 87.75%
Current values at position (1896, 4096):
  25th percentile: 2.148855149745941
  Med

Processing models  75%|██████████████████████████        |  ETA: 3:01:40

Processing model 19 of 24: cama-flood_matsiro_ipsl-cm5a-lr_ewembi_rcp26_2005soc_co2_flddph_none_150arcsec_global_annual_2006_2100.nc4

Status Update:
Progress: 27.53%
Current values at position (595, 5411):
  25th percentile: 0.0
  Median: 0.0
  75th percentile: 0.49764490127563477
  Theil-Sen slope: 0.0
  Correlation p-value: 0.29569413170270537


Status Update:
Progress: 38.60%
Current values at position (834, 6390):
  25th percentile: 3.2024168968200684
  Median: 4.348371505737305
  75th percentile: 4.861316561698914
  Theil-Sen slope: -0.009204089641571045
  Correlation p-value: 0.019512234110912414


Status Update:
Progress: 55.21%
Current values at position (1193, 4125):
  25th percentile: 3.1339536905288696
  Median: 4.171262502670288
  75th percentile: 7.2110161781311035
  Theil-Sen slope: 0.012175559997558594
  Correlation p-value: 0.3541738045208428


Status Update:
Progress: 88.36%
Current values at position (1909, 5066):
  25th percentile: 0.0
  Median: 0.0
  75th percentil

Processing models  79%|███████████████████████████       |  ETA: 2:31:11

In [ ]:
# Required packages
using NCDatasets
using Statistics
using Dates
using DataFrames
using StatsBase
using Plots
using ProgressMeter
using HypothesisTests
using Printf
using Base.Threads

"""
Process NetCDF models with chunked latitude processing
"""
function process_models_chunked_lat(directory_path::String, rcp_value::String, return_period_value::String)
    # List all files in the directory
    all_files = readdir(directory_path, join=true)
    println("Total files found in directory: ", length(all_files))
    
    # Filter for .nc4 files
    nc4_files = filter(f -> endswith(f, ".nc4"), all_files)
    println("Files with .nc4 extension: ", length(nc4_files))
    
    # Split each filename by underscore and check the specific positions
    matching_files = filter(nc4_files) do f
        parts = split(basename(f), "_")
        # Check if we have enough parts and the specific positions match
        length(parts) ≥ 9 && 
        parts[5] == rcp_value && 
        parts[9] == return_period_value
    end
    
    println("Files matching RCP '$(rcp_value)' and return period '$(return_period_value)': ", length(matching_files))
    if !isempty(matching_files)
        println("Example matching file: ", basename(matching_files[1]))
    end
    
    isempty(matching_files) && error("No files found matching criteria. Please check the directory path and file naming pattern.")
 
        
    # Get dimensions from first file
    ds = Dataset(matching_files[1])
    time_var = ds["time"][:]  # Changed from 'time' to 'time_var'
    lat = ds["lat"][:]
    lon = ds["lon"][:]
    close(ds)

    num_models = length(matching_files)
    num_time = length(time_var)  # Changed from 'time' to 'time_var'
    num_lat = length(lat)
    num_lon = length(lon)

    # Set chunk size
    chunk_size_lat = ceil(Int, num_lat / 4)

     # Initialize result array with threads
    result = Array{Float64}(undef, num_lat, num_lon, 5, num_models)
    
    # Process each model
    @showprogress "Processing models" for (model_idx, file) in enumerate(matching_files)
        println("Processing model $model_idx of $num_models: $(basename(file))")
        ds = Dataset(file)
        
        # Process latitude chunks in parallel
        lat_indices = 1:chunk_size_lat:num_lat
        
        @threads for (chunk_idx, lat_start) in collect(enumerate(lat_indices))
            lat_end = min(lat_start + chunk_size_lat - 1, num_lat)
            lat_count = lat_end - lat_start + 1
            
            # Read chunk data
            flddph_chunk = permutedims(
                ds["flddph"][:, lat_start:lat_end, :],
                [2, 1, 3]
            )
            
            # Process each grid point in chunk
            for i in 1:lat_count
                for j in 1:num_lon
                    data_ts = flddph_chunk[i, j, :]
                    
                    # Convert Missing to Float64 and handle NaN
                    data_ts_float = Float64[ismissing(x) ? NaN : Float64(x) for x in data_ts]
                    valid_idx = findall(!isnan, data_ts_float)
                    valid_data = data_ts_float[valid_idx]
                    
                    if length(valid_data) >= 2
                        # Calculate quantiles regardless of uniqueness
                        first_40 = valid_data[1:min(40, length(valid_data))]
                        result[lat_start+i-1, j, 1, model_idx] = quantile(first_40, 0.25)
                        result[lat_start+i-1, j, 2, model_idx] = quantile(first_40, 0.50)
                        result[lat_start+i-1, j, 3, model_idx] = quantile(first_40, 0.75)
                        
                        if length(unique(valid_data)) > 1
                            # Calculate slope and correlation only if there's variability
                            slopes = [(valid_data[i2] - valid_data[i1]) / (valid_idx[i2] - valid_idx[i1])
                                    for i1 in 1:length(valid_idx)
                                    for i2 in (i1+1):length(valid_idx)]
                            
                            result[lat_start+i-1, j, 4, model_idx] = median(slopes)
                            test = CorrelationTest(Float64.(valid_idx), valid_data)
                            result[lat_start+i-1, j, 5, model_idx] = pvalue(test)
                        else
                            result[lat_start+i-1, j, 4:5, model_idx] .= NaN
                        end
                    else
                        result[lat_start+i-1, j, :, model_idx] .= NaN
                    end
                end
            end
        end
        close(ds)
    end
    return result, lat, lon
end

"""
Save results to NetCDF file
"""
function save_to_netcdf(result, lat, lon, output_path, rcp_value, return_period_value)
    ds = Dataset(output_path, "c")
    
    # Define dimensions
    defDim(ds, "lat", length(lat))
    defDim(ds, "lon", length(lon))
    defDim(ds, "value", 5)
    defDim(ds, "model", size(result, 4))
    
    # Define variables
    v = defVar(ds, "result", Float64, ("lat", "lon", "value", "model"))
    v[:,:,:,:] = result
    
    # Add attributes
    ds.attrib["title"] = "Results Array Containing Statistical Analysis"
    ds.attrib["creation_date"] = string(now())
    
    close(ds)
end

# Main execution
function main()
    nc_in_path = "C:\\ClimateData\\Flood\\FloodDepth\\Future\\rawNcs\\"
    nc_out_path = "C:\\ClimateData\\Flood\\FloodDepth\\Reanalysis\\"
    rcp_value = "rcp26"
    return_period_value = "none"
    
    start_time = now()
    
    # Process data
    result, lat, lon = process_models_chunked_lat(nc_in_path, rcp_value, return_period_value)
    
    # Save results
    output_file = joinpath(nc_out_path, "results_$(rcp_value)_$(return_period_value).nc4")
    save_to_netcdf(result, lat, lon, output_file, rcp_value, return_period_value)
    
    println("Time elapsed: ", now() - start_time)
end

# Run the program
main()

Total files found in directory: 216
Files with .nc4 extension: 216
Files matching RCP 'rcp26' and return period 'none': 24
Example matching file: cama-flood_clm45_gfdl-esm2m_ewembi_rcp26_2005soc_co2_flddph_none_150arcsec_global_annual_2006_2100.nc4

In [ ]:
1-1

In [ ]:
# for multithreading use: julia --threads auto

In [6]:
using NCDatasets
using Statistics
using StatsBase
using HypothesisTests
using Dates
using ProgressMeter

"""
Compute Theil-Sen slope estimator
"""
function theil_sen_slope(x::Vector{T}, y::Vector{T}) where T<:Real
    n = length(x)
    slopes = Vector{Float64}()
    sizehint!(slopes, div(n * (n-1), 2))
    
    for i in 1:n
        for j in (i+1):n
            if x[i] != x[j]  # Avoid division by zero
                push!(slopes, (y[j] - y[i]) / (x[j] - x[i]))
            end
        end
    end
    
    return isempty(slopes) ? NaN : median(slopes)
end

"""
Process NetCDF files with chunked latitude processing
"""
function process_models_chunked_lat(directory_path::String, rcp_value::String, return_period_value::String)
    # Get list of files matching criteria
    files = filter(f -> occursin("_$(rcp_value)_.*_$(return_period_value)_", f) && endswith(f, ".nc4"),
                  readdir(directory_path, join=true))
    
    isempty(files) && error("No files found matching the specified RCP and return period criteria")
    
    # Open first file to get dimensions
    ds = Dataset(files[1])
    time = ds["time"][:]
    lat = ds["lat"][:]
    lon = ds["lon"][:]
    close(ds)
    
    num_models = length(files)
    num_time = length(time)
    num_lat = length(lat)
    num_lon = length(lon)
    
    # Initialize result array with named dimensions
    result = Array{Float64}(undef, num_lat, num_lon, 5, num_models)
    fill!(result, NaN)
    
    # Process each model
    @showprogress "Processing models" for (model_idx, file) in enumerate(files)
        ds = Dataset(file)
        chunk_size_lat = ceil(Int, num_lat / 4)
        
        # Process in latitude chunks
        for lat_start in 1:chunk_size_lat:num_lat
            lat_end = min(lat_start + chunk_size_lat - 1, num_lat)
            lat_count = lat_end - lat_start + 1
            
            # Read chunk data
            flddph_chunk = permutedims(
                ds["flddph"][1:num_lon, lat_start:lat_end, 1:num_time],
                [2, 1, 3]
            )
            
            # Process each grid point
            @threads for i in 1:lat_count
                for j in 1:num_lon
                    data_ts = @view flddph_chunk[i, j, :]
                    valid_mask = .!isnan.(data_ts)
                    valid_data = data_ts[valid_mask]
                    
                    if length(valid_data) >= 2 && length(unique(valid_data)) > 1
                        # Calculate percentiles
                        result[lat_start+i-1, j, 1, model_idx] = quantile(valid_data[1:min(40,end)], 0.25)
                        result[lat_start+i-1, j, 2, model_idx] = quantile(valid_data[1:min(40,end)], 0.50)
                        result[lat_start+i-1, j, 3, model_idx] = quantile(valid_data[1:min(40,end)], 0.75)
                        
                        # Calculate Theil-Sen slope
                        x = collect(1:length(valid_data))
                        result[lat_start+i-1, j, 4, model_idx] = theil_sen_slope(x, valid_data)
                        
                        # Calculate Spearman correlation
                        result[lat_start+i-1, j, 5, model_idx] = pvalue(SpearmanCorTest(x, valid_data))
                    elseif length(valid_data) >= 1
                        # Handle cases with single value
                        result[lat_start+i-1, j, 1:3, model_idx] .= valid_data[1]
                        result[lat_start+i-1, j, 4:5, model_idx] .= NaN
                    end
                end
            end
        end
        close(ds)
        GC.gc()  # Optional garbage collection
    end
    
    return (result=result, lat=lat, lon=lon)
end

"""
Save results to NetCDF file
"""
function save_results_to_netcdf(result, lat, lon, output_path, rcp_value, return_period_value)
    ds = Dataset(output_path, "c")
    
    # Define dimensions
    defDim(ds, "lat", length(lat))
    defDim(ds, "lon", length(lon))
    defDim(ds, "value", 5)
    defDim(ds, "model", size(result, 4))
    
    # Define variables
    v = defVar(ds, "result", Float64, ("lat", "lon", "value", "model"))
    v.attrib["long_name"] = "Statistical results"
    v.attrib["missing_value"] = NaN
    
    # Define coordinate variables
    lat_var = defVar(ds, "lat", Float64, ("lat",))
    lon_var = defVar(ds, "lon", Float64, ("lon",))
    
    value_names = ["p25", "p50", "p75", "theil_sen_slope", "spearman_p"]
    value_var = defVar(ds, "value_names", String, ("value",))
    
    # Write data
    v[:,:,:,:] = result
    lat_var[:] = lat
    lon_var[:] = lon
    value_var[:] = value_names
    
    # Add global attributes
    ds.attrib["title"] = "Statistical Analysis Results"
    ds.attrib["creation_date"] = string(Dates.now())
    ds.attrib["rcp_value"] = rcp_value
    ds.attrib["return_period_value"] = return_period_value
    
    close(ds)
end

"""
Visualize results using Plots
"""
function visualize_results(nc_file_path)
    using Plots
    
    ds = Dataset(nc_file_path)
    lat = ds["lat"][:]
    lon = ds["lon"][:]
    data = ds["result"][:,:,1,1]  # First statistic, first model
    close(ds)
    
    # Flip latitude if needed
    if lat[1] > lat[end]
        lat = reverse(lat)
        data = reverse(data, dims=1)
    end
    
    heatmap(lon, lat, data',
            xlabel="Longitude",
            ylabel="Latitude",
            title="Results Visualization",
            color=:viridis)
end



LoadError: LoadError: UndefVarError: `@threads` not defined
in expression starting at c:\Users\arikt\Documents\GitHub\juliaAndBigData\ClimateHazards\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W1sZmlsZQ==.jl:27

In [5]:
# First, make sure you have all required packages
using Pkg
Pkg.add(["NCDatasets", "Statistics", "StatsBase", "HypothesisTests", "Dates", "ProgressMeter", "Plots"])

# Import the packages
using NCDatasets
using Statistics
using StatsBase
using HypothesisTests
using Dates
using ProgressMeter
using Plots

# Set your parameters
main_directory_path = "C:/ClimateData/Flood/FloodDepth/"
input_directory_path = "$(main_directory_path)Future/RawNcs/"
rcp_value = "rcp45"                           # Replace with your RCP value
return_period_value = "none"                 # Replace with your return period
output_path = "$(main_directory_path)Reanalysis/results_$(rcp_value)_$(return_period_value).nc"

# Process the data
@time begin
    results = process_models_chunked_lat(directory_path=input_directory_path, rcp_value, return_period_value)
end

# Save the results to NetCDF
save_results_to_netcdf(results.result, results.lat, results.lon,
                      output_path, rcp_value, return_period_value)

# Visualize the results
visualize_results(output_path)

   Resolving package versions...
  No Changes to `C:\Users\arikt\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\arikt\.julia\environments\v1.10\Manifest.toml`


UndefVarError: UndefVarError: `process_models_chunked_lat` not defined

In [ ]:
# Example usage:
function main()
    directory_path = "path_to_nc_files"
    rcp_value = "rcp45"
    return_period_value = "100yr"
    output_path = "results_$(rcp_value)_$(return_period_value).nc"
    
    # Process data
    @time begin
        results = process_models_chunked_lat(directory_path, rcp_value, return_period_value)
    end
    
    # Save results
    save_results_to_netcdf(results.result, results.lat, results.lon,
                          output_path, rcp_value, return_period_value)
    
    # Visualize results
    visualize_results(output_path)
end